In [53]:
# Calculating the missing percentages of all missing variables in data
import pandas as pd

"""Temp!!!"""
df_encoded = pd.read_pickle("df_encoded_files/df_encoded.pkl")


# Define the columns to replace NaN values with 0
columns_to_replace = [
    "SYSTOLIC_BP_min", "SYSTOLIC_BP_max", "SYSTOLIC_BP_mean",
    "DIASTOLIC_BP_min", "DIASTOLIC_BP_max", "DIASTOLIC_BP_mean"
]

# Replace NaN values with 0 in the specified columns
df_encoded[columns_to_replace] = df_encoded[columns_to_replace].fillna(0)

# Compute the percentage of missing values (i.e., values that are 0) for the specified columns
missing_percentages = (df_encoded[columns_to_replace] == 0).mean() * 100

# Display the missing percentages
print("Missing percentages (based on 0 values) for each column:")
print(missing_percentages)

# List of variables to check
variables_to_check_0 = [
    'min_BMI', 'min_HEIGHT', 'min_PULSE', 'min_WEIGHT', 
    'max_BMI', 'max_HEIGHT', 'max_PULSE', 'max_WEIGHT', 
    'mean_BMI', 'mean_HEIGHT', 'mean_PULSE', 'mean_WEIGHT',
    'SYSTOLIC_BP_min', 'SYSTOLIC_BP_max', 'SYSTOLIC_BP_mean',
    'DIASTOLIC_BP_min', 'DIASTOLIC_BP_max', 'DIASTOLIC_BP_mean',


]

variables_to_check_1 = [
    'PRIMARY_RACE_Unknown', 
    'LANGUAGE_Unknown',
    'PRIMARY_ETHNICITY_Unknown',
    'D_Insur_at_pull_Unknown',
]

# Calculate missing percentages (where value == 0)
missing_percentages = (df_encoded[variables_to_check_0] == 0).mean() * 100

# Calculate missing percentages (where value == 1)
missing_percentages = (df_encoded[variables_to_check_1] == 1).mean() * 100


# Convert to a DataFrame for better visualization
missing_df = missing_percentages.reset_index()
missing_df.columns = ['Variable', 'Missing_Percentage']

# Print the missing percentages
print(missing_df)


"""

Variable  Missing_Percentage
0       PRIMARY_RACE_Unknown            5.985169
1           LANGUAGE_Unknown            0.123588
2  PRIMARY_ETHNICITY_Unknown           20.533192
3    D_Insur_at_pull_Unknown            0.052966

0             min_BMI            1.906780
1          min_HEIGHT            1.553672
2           min_PULSE            2.454096
3          min_WEIGHT            0.141243
4             max_BMI            1.889124
5          max_HEIGHT            1.553672
6           max_PULSE            0.017655
7          max_WEIGHT            0.141243
8            mean_BMI            1.889124
9         mean_HEIGHT            1.553672
10         mean_PULSE            0.017655
11        mean_WEIGHT            0.141243

"""

Missing percentages (based on 0 values) for each column:
SYSTOLIC_BP_min      3.684608
SYSTOLIC_BP_max      3.684608
SYSTOLIC_BP_mean     3.684608
DIASTOLIC_BP_min     3.684608
DIASTOLIC_BP_max     3.684608
DIASTOLIC_BP_mean    3.684608
dtype: float64
                    Variable  Missing_Percentage
0       PRIMARY_RACE_Unknown            7.230885
1           LANGUAGE_Unknown            0.213783
2  PRIMARY_ETHNICITY_Unknown           26.810865
3    D_Insur_at_pull_Unknown            0.968310


'\n\nVariable  Missing_Percentage\n0       PRIMARY_RACE_Unknown            5.985169\n1           LANGUAGE_Unknown            0.123588\n2  PRIMARY_ETHNICITY_Unknown           20.533192\n3    D_Insur_at_pull_Unknown            0.052966\n\n0             min_BMI            1.906780\n1          min_HEIGHT            1.553672\n2           min_PULSE            2.454096\n3          min_WEIGHT            0.141243\n4             max_BMI            1.889124\n5          max_HEIGHT            1.553672\n6           max_PULSE            0.017655\n7          max_WEIGHT            0.141243\n8            mean_BMI            1.889124\n9         mean_HEIGHT            1.553672\n10         mean_PULSE            0.017655\n11        mean_WEIGHT            0.141243\n\n'

In [54]:
import pandas as pd

df_encoded = pd.read_pickle("df_encoded_files/df_encoded.pkl")

# Convert boolean data types to integers
df_encoded = df_encoded.astype({col: int for col in df_encoded.select_dtypes(include=['bool']).columns})

# Identify columns that start with 'PRIMARY_RACE' but exclude 'PRIMARY_RACE_Unknown'
race_columns = [col for col in df_encoded.columns if col.startswith("PRIMARY_RACE") and col != "PRIMARY_RACE_Unknown"]

# Set 'PRIMARY_RACE_Unknown' to 1 if all other 'PRIMARY_RACE' columns are 0
df_encoded.loc[df_encoded[race_columns].sum(axis=1) == 0, "PRIMARY_RACE_Unknown"] = 1

unknown_ethnicity_list = []

# Filter the dataframe to get rows where PRIMARY_ETHNICITY_Unknown == 1
filtered_rows = df_encoded[df_encoded['PRIMARY_ETHNICITY_Unknown'] == 1]

i = 0
# Print the row index (from df) along with the ID column
for index, row in filtered_rows.iterrows():
    # if i <= 25:
    #     print(f"Row Index: {index}, ID: {row['ID']}")
    #     i += 1
    unknown_ethnicity_list.append(index)


unknown_race_list = []

# Filter the dataframe to get rows where PRIMARY_RACE_Unknown == 1
filtered_rows = df_encoded[df_encoded['PRIMARY_RACE_Unknown'] == 1]

# print('='*50)
i = 0
# Print the row index (from df) along with the ID column
for index, row in filtered_rows.iterrows():
    # if i <= 25:
    #     print(f"Row Index: {index}, ID: {row['ID']}")
    #     i += 1
    unknown_race_list.append(index)

# List of columns that need to be converted
columns_to_convert = [
    'PRIMARY_RACE_American Indian / Native American',
    'PRIMARY_RACE_Asian',
    'PRIMARY_RACE_Asian Indian',
    'PRIMARY_RACE_Black / African American',
    'PRIMARY_RACE_Middle Eastern',
    'PRIMARY_RACE_Native Hawaiian / Pacific Islander',
    'PRIMARY_RACE_White'
]

# Ensure only existing columns are processed
existing_columns = [col for col in columns_to_convert if col in df_encoded.columns]

# Convert string-based boolean values to actual integers (0 and 1), handling NaNs
df_encoded[existing_columns] = df_encoded[existing_columns].apply(
    lambda col: col.astype(str).str.lower().map({'false': 0, 'true': 1}).fillna(0).astype(int)
)

# Verify that conversion was successful
print("✅ Specified object-type columns successfully converted to 0 and 1.")
print(df_encoded[existing_columns].dtypes)  # Should all be int64 or int32

df_encoded["RPL_THEME1"].fillna(0, inplace=True)

✅ Specified object-type columns successfully converted to 0 and 1.
PRIMARY_RACE_American Indian / Native American     int64
PRIMARY_RACE_Asian                                 int64
PRIMARY_RACE_Asian Indian                          int64
PRIMARY_RACE_Black / African American              int64
PRIMARY_RACE_Middle Eastern                        int64
PRIMARY_RACE_Native Hawaiian / Pacific Islander    int64
PRIMARY_RACE_White                                 int64
dtype: object


In [55]:
"""
Impute datas are are barely missing
"""

from sklearn.impute import KNNImputer

# Define the columns to impute
columns_to_impute = [
    "min_BMI", "min_HEIGHT", "min_PULSE", "min_WEIGHT",
    "max_BMI", "max_HEIGHT", "max_PULSE", "max_WEIGHT",
    "mean_BMI", "mean_HEIGHT", "mean_PULSE", "mean_WEIGHT",
    "SYSTOLIC_BP_min", "SYSTOLIC_BP_max", "SYSTOLIC_BP_mean",
    "DIASTOLIC_BP_min", "DIASTOLIC_BP_max", "DIASTOLIC_BP_mean"
]

# Create a copy of the dataset to avoid modifying the original
df_knn_impute = df_encoded.copy()

# Initialize KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)

# Apply KNN Imputation only to selected columns
df_knn_impute[columns_to_impute] = knn_imputer.fit_transform(df_knn_impute[columns_to_impute])

# Replace only the NaN values in df_encoded with the imputed values
df_encoded[columns_to_impute] = df_encoded[columns_to_impute].where(df_encoded.notna(), df_knn_impute[columns_to_impute])


In [56]:
# Check for 0s
zero_count = (df_encoded["RPL_THEME1"] == 0).sum()

# Check for blank (NaN) values
nan_count = df_encoded["RPL_THEME1"].isna().sum()

# Print results
print(f"Number of 0s in RPL_THEME1: {zero_count}")
print(f"Number of blank (NaN) values in RPL_THEME1: {nan_count}")


Number of 0s in RPL_THEME1: 1152
Number of blank (NaN) values in RPL_THEME1: 0


In [58]:
import torch
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import warnings
import torch

# Suppress FutureWarnings globally
warnings.simplefilter(action='ignore', category=FutureWarning)


# 🔹 I-NAAutoencoder Model with Undercomplete Representation and Dropout
class INA_Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(INA_Autoencoder, self).__init__()

        # Undercomplete Representation: Smaller hidden layers than input
        hidden_dim1 = int(input_dim * 0.75)  # Compress to 75% of input size
        hidden_dim2 = int(input_dim * 0.60)  # Compress to 60% of input size
        hidden_dim3 = int(input_dim * 0.5)   # Compress to 50% of input size
        

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim3, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim2, hidden_dim1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim1, input_dim),
            nn.Sigmoid()  # Output between 0 and 1 for both continuous and binary features
        )

    def forward(self, x):
        if torch.isnan(x).any():
            print("NaN detected in input")
        
        encoded = self.encoder(x)
        
        if torch.isnan(encoded).any():
            print("NaN detected in encoding")

        decoded = self.decoder(encoded)
        
        if torch.isnan(decoded).any():
            print("NaN detected in output")
        
        return decoded


# Define the columns to remove before imputation
unknown_columns = [
    'PRIMARY_ETHNICITY_Unknown',
    'PRIMARY_RACE_Unknown',
    'LANGUAGE_Unknown',
    'D_Insur_at_pull_Unknown',
]

# 🔹 Create a copy of df_encoded to avoid modifying original data
df_impute = df_encoded.drop(columns=unknown_columns, errors='ignore').copy()
df_impute = df_impute.drop(columns=['ID'], errors='ignore')


# 🔹 Load the trained I-NAA model
model_path = "best_inaa_models/best_INAA_MAR_30.pth"


# Initialize the model with the correct input size (based on df_impute)
input_dim = df_impute.shape[1]
model = INA_Autoencoder(input_dim)  # Ensure the model matches feature count
model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 🔹 Identify rows where any of the _Unknown columns are 1 (indicating missing categorical values)
missing_rows = (df_encoded['RPL_THEME1'] == 0) | df_encoded['RPL_THEME1'].isna()

# 🔹 Prepare data for imputation
input_data = torch.tensor(df_impute.loc[missing_rows].values, dtype=torch.float32).to(device)

# 🔹 Perform imputation with the trained model
with torch.no_grad():
    imputed_data = model(input_data).cpu().numpy()  # Move back to CPU

# 🔹 Replace only the missing rows in the original dataset with imputed values
df_impute.loc[missing_rows, df_impute.columns] = imputed_data

# ✅ Successfully imputed missing categorical values while keeping _Unknown columns intact!
print("✅ Imputation complete! _Unknown columns are preserved.")

df_encoded.loc[missing_rows, "RPL_THEME1"] = df_impute.loc[missing_rows, "RPL_THEME1"]

✅ Imputation complete! _Unknown columns are preserved.


In [ ]:
df_encoded.to_pickle("df_encoded_imputed_final.pkl")

In [ ]:
"""

Once imputation training is done, we first run all the cells like usual and run the imputation model that is 30% missing on MAR 
use that to impute our ethnicities. after that just rerun the training code cell but change the model to use the one that is 10% 
missing on MAR. do the same thing but this time we change the values for race. after we impute all our values into df_encoded we 
save it as df_encoded_imputed.pkl

"""